In [ ]:
from google.colab import files

uploaded = files.upload()


In [14]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 68 kB 2.6 MB/s 
     |████████████████████████████████| 86 kB 2.7 MB/s 
     |████████████████████████████████| 140 kB 34.0 MB/s 
     |████████████████████████████████| 212 kB 38.5 MB/s 
     |████████████████████████████████| 346 kB 31.0 MB/s 
     |████████████████████████████████| 1.1 MB 39.2 MB/s 
     |████████████████████████████████| 596 kB 36.1 MB/s 
     |████████████████████████████████| 127 kB 43.0 MB/s 
     |████████████████████████████████| 144 kB 9.0 MB/s 
     |████████████████████████████████| 271 kB 45.4 MB/s 
     |████████████████████████████████| 94 kB 2.4 MB/s 
     |████████████████████████████████| 86 kB 5.1 MB/s 
     |████████████████████████████████| 112 kB 51.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24

In [15]:
import evaluate

In [ ]:
## load transformer and initialize

!pip install transformers==3.1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 884 kB 5.1 MB/s 
     |████████████████████████████████| 880 kB 23.6 MB/s 
     |████████████████████████████████| 1.2 MB 55.3 MB/s 
     |████████████████████████████████| 3.0 MB 30.4 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=c832a16474a445bd1b46c3ad7528787f8643eafc3950ab4239a139a736be0f32
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses


In [1]:
from transformers import pipeline
#initialize roberta-large model
classifier = pipeline("zero-shot-classification", model='joeddav/xlm-roberta-large-xnli')

In [ ]:
# initialize facebook model
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

The following experiment utilizes two corpus dataset:
- 200 English Tweets (100 labelled hateful tweets and 100 labelled non-hateful tweets)
- 200 Swahili Tweets (100 labelled hateful tweets and 100 labelled non-hatefull tweets)

We can use this pipeline by passing in a sequence and a list of candidate labels. The pipeline assumes by default that only one of the candidate labels is true, returning a list of scores for each label which add up to 1.

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving en_200_examples.xlsx - en_200_examples.csv to en_200_examples.xlsx - en_200_examples.csv


In [4]:
## read 200 samples of curated English tweets
import pandas as pd
eng_tweets = pd.read_csv("en_200_examples.xlsx - en_200_examples.csv")
eng_tweets

,Gold Standard ID,Language,X,text,user_name,user_id,label
0,gs201,English,55378,@johnnjenga @BUANTAI004 @ShaffieWeru @mithika_...,Lisimba,3918809081,hateful
1,gs202,English,9717,How many Kikuyu spokesmen do we need in Mt. Ke...,"Peter Kariuki, M.Sc.",1390948818129203204,hateful
2,gs203,English,50071,@johnnjenga @ShaffieWeru @mithika_Linturi @SAB...,MkisiiWaNairobi,1352158178423828480,hateful
3,gs204,English,57783,Today’s “vijana” are the smartest but the lazi...,Rodney Musalia,839334020,hateful
4,gs205,English,46128,"In Azimio we only have two women, @gladyswanga...",Sir Breezy,1236327598214561793,hateful
...,...,...,...,...,...,...,...
195,gs396,English,27938,RT @Hon_FatumaGedi: Another achievement am hum...,Victor Alago,2242296609,non-hateful
196,gs397,English,45211,RT @HonTimWanyonyi: I have taken note of the m...,Padwick Maratani,286317898,non-hateful
197,gs398,English,8300,RT @HonKimariKihara: At Gîtugî Mixed secondary...,James Kihara,424701665,non-hateful
198,gs399,English,14270,@KoechNelsonK Fuel prices at the background ke...,VICTOR MIBEI,607315601,non-hateful


In [5]:
subset = eng_tweets[["Language","text","label"]]
subset.head(3)

,Language,text,label
0,English,@johnnjenga @BUANTAI004 @ShaffieWeru @mithika_...,hateful
1,English,How many Kikuyu spokesmen do we need in Mt. Ke...,hateful
2,English,@johnnjenga @ShaffieWeru @mithika_Linturi @SAB...,hateful


In [ ]:
# equal samples for each label
subset.label.value_counts()

hateful        100
non-hateful    100
Name: label, dtype: int64

In [6]:
# split subset along the non-hateful/hateful labels with equal samples

eng_hateful_tweets = subset.iloc[0:100]
eng_non_hateful = subset.iloc[100:200]
eng_non_hateful.tail(3)

,Language,text,label
197,English,RT @HonKimariKihara: At Gîtugî Mixed secondary...,non-hateful
198,English,@KoechNelsonK Fuel prices at the background ke...,non-hateful
199,English,RT @Moha001_Onyango: Haha i have boarded this ...,non-hateful


In [84]:
%%time
# train model english hateful tweets
# loop through the text (tweets) col
scores = []
candidate_labels = ["hateful", "loving"]
hypothesis_template = "this sentence is {}."

for sequence in eng_hateful_tweets.text:
  # without multiclass
  score = classifier(sequence, candidate_labels, hypothesis_template)
  scores.append(score)
df = pd.DataFrame.from_dict(scores)
df.reset_index(drop=True)

corpus_1 = df["labels"].apply(pd.Series)
corpus_2 = df["scores"].apply(pd.Series)

corpus_1.reset_index()
corpus_1.rename(columns = {0:'label_one'}, inplace = True)
corpus_1.rename(columns = {1:'label_two'}, inplace = True)
corpus_2.rename(columns = {0:'score_one'}, inplace = True)
corpus_2.rename(columns = {1:'score_two'}, inplace = True)
corpus = pd.concat([corpus_1, corpus_2], axis=1)

# restructure subset
plot = corpus.groupby(["label_one","label_two"])["score_one","score_two"].mean().reset_index()
new_plot = plot[["label_one", "score_one", "label_two", "score_two"]]

CPU times: user 2min 17s, sys: 722 ms, total: 2min 18s
Wall time: 2min 17s


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [ ]:
new_plot

,label_one,score_one,label_two,score_two
0,hateful,0.901933,loving,0.098067
1,loving,0.718302,hateful,0.281698


In [86]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=2, cols=1, shared_yaxes=True, shared_xaxes=True, subplot_titles=('True Positive/Negative',  'False Positive/Negative'))

fig.add_trace(go.Bar(y=new_plot["label_one"], 
                     x=new_plot["score_one"],
                     name = "True Positive/Negative on English (Hateful Tweets)",
                     orientation = "h",
                     marker=dict(color=[4, 5, 6], coloraxis="coloraxis")),
              1, 1)
fig.add_trace(go.Bar(y=new_plot["label_two"], 
                     x=new_plot["score_two"],
                     name = "False Positive/Negative Prediction on English (Hateful Tweets)",
                     orientation = "h",
                     marker=dict(color=[4, 5, 6], coloraxis="coloraxis")),
              2, 1)


fig.update_layout(title = "Performance of xlm-roberta-large-xnli on English Hateful Tweets",
                  xaxis_title = "Model Score",
                  yaxis_title = "Class labels",
                  coloraxis=dict(colorscale='Bluered_r'), 
                  plot_bgcolor = "white",
                  barmode = "group",
                  showlegend=False)
fig.show()

In [7]:
%%time
# user-defined function allows three parameters
# allow user to choose to include a template
# allow user to choose multi-class approach of classification - explain.
scores = []
candidate_labels = ["hateful", "loving"]
hypothesis_template = "this sentence is {}."

for sequence in eng_non_hateful.text:
  # without multiclass
  score = classifier(sequence, candidate_labels, hypothesis_template)
  scores.append(score)
df = pd.DataFrame.from_dict(scores)
df.reset_index(drop=True)

corpus_1 = df["labels"].apply(pd.Series)
corpus_2 = df["scores"].apply(pd.Series)

corpus_1.reset_index()
corpus_1.rename(columns = {0:'label_one'}, inplace = True)
corpus_1.rename(columns = {1:'label_two'}, inplace = True)
corpus_2.rename(columns = {0:'score_one'}, inplace = True)
corpus_2.rename(columns = {1:'score_two'}, inplace = True)
corpus = pd.concat([corpus_1, corpus_2], axis=1)

CPU times: user 2min 14s, sys: 589 ms, total: 2min 15s
Wall time: 2min 14s


In [49]:
corpus

,label_one,label_two,score_one,score_two
0,loving,hateful,0.842211,0.157789
1,loving,hateful,0.867749,0.132251
2,loving,hateful,0.727847,0.272153
3,loving,hateful,0.620586,0.379414
4,hateful,loving,0.579634,0.420366
...,...,...,...,...
95,loving,hateful,0.978550,0.021450
96,loving,hateful,0.772432,0.227568
97,loving,hateful,0.682730,0.317270
98,hateful,loving,0.714498,0.285502


In [8]:
plot = corpus.groupby(["label_one","label_two"])["score_one","score_two"].mean().reset_index()
new_plot = plot[["label_one", "score_one", "label_two", "score_two"]]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


In [9]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=2, cols=1, shared_yaxes=True, shared_xaxes=True, subplot_titles=('True Positive/Negative',  'False Positive/Negative'))

fig.add_trace(go.Bar(y=new_plot["label_one"], 
                     x=new_plot["score_one"],
                     name = "True Positive/Negative on English (Non-Hateful Tweets)",
                     orientation = "h",
                     marker=dict(color=[4, 5, 6], coloraxis="coloraxis")),
              1, 1)
fig.add_trace(go.Bar(y=new_plot["label_two"], 
                     x=new_plot["score_two"],
                     name = "False Positive/Negative Prediction on English (Non-Hateful Tweets)",
                     orientation = "h",
                     marker=dict(color=[4, 5, 6], coloraxis="coloraxis")),
              2, 1)


fig.update_layout(title = "Performance of xlm-roberta-large-xnli on English Non-hateful Tweets",
                  xaxis_title = "Model Score",
                  yaxis_title = "Class labels",
                  coloraxis=dict(colorscale='Bluered_r'), 
                  plot_bgcolor = "white",
                  barmode = "group",
                  showlegend=False)
fig.show()

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving sw_200_examples.xlsx - sw_200_examples.csv to sw_200_examples.xlsx - sw_200_examples.csv


In [20]:
swa_tweets = pd.read_csv("sw_200_examples.xlsx - sw_200_examples.csv")
swa_tweets = swa_tweets[["Language", "text", "label"]]
swa_tweets.head()

,Language,text,label
0,Kiswahili,#KenyaKwanza,hateful
1,Kiswahili,@HassanAliJoho WAtoto Mombasa wamekushinda eas...,hateful
2,Kiswahili,Rudi nyumbani dada mwajuma 🤣🤣🤣🤣 @HonKangata,hateful
3,Kiswahili,Wanalipa watu kumobilize zii Kakamega ni Azimi...,hateful
4,Kiswahili,@MoiGideon Yaani Mheshimiwa wewe imeonekana u ...,hateful


In [ ]:
swa_tweets.label.value_counts()

hateful        100
non-hateful    100
Name: label, dtype: int64

In [ ]:
swa_hateful_tweets = swa_tweets.iloc[0:100]
swa_non_hateful = swa_tweets.iloc[100:200]
swa_non_hateful

,Language,text,label
100,Kiswahili,RT @PierreTole: Nairobi ni @HonTimWanyonyi na ...,non-hateful
101,Kiswahili,RT @LenonRotich: This race gonna be like Marat...,non-hateful
102,Kiswahili,RT @Daisykenyan: @anc_party Kenya kwanza is un...,non-hateful
103,Kiswahili,"RT @AzimioTv: Ugunja, #AzimioMtaaniSiaya https...",non-hateful
104,Kiswahili,RT @Hon_Kagesi: Asante sana Vihiga \n\nKenya K...,non-hateful
...,...,...,...
195,Kiswahili,RT @revolutionery21: @anc_party Kenya Kwanza i...,non-hateful
196,Kiswahili,RT @shimolatewa001: @WilliamsRuto @HonJumwa\n@...,non-hateful
197,Kiswahili,"RT @anc_party: Kenya Kwanza, In Butula, Busia ...",non-hateful
198,Kiswahili,"RT @anc_party: Kenya Kwanza, In Butula, Busia ...",non-hateful


In [68]:
%%time
# loop through the text (tweets) col
scores = []
candidate_labels = ["hateful", "loving"]
hypothesis_template = "hii sentensi ni{}."

for sequence in swa_hateful_tweets.text:
  # without multiclass
  score = classifier(sequence, candidate_labels, hypothesis_template)
  scores.append(score)
df = pd.DataFrame.from_dict(scores)
df.reset_index(drop=True)

corpus_1 = df["labels"].apply(pd.Series)
corpus_2 = df["scores"].apply(pd.Series)

corpus_1.reset_index()
corpus_1.rename(columns = {0:'label_one'}, inplace = True)
corpus_1.rename(columns = {1:'label_two'}, inplace = True)
corpus_2.rename(columns = {0:'score_one'}, inplace = True)
corpus_2.rename(columns = {1:'score_two'}, inplace = True)
corpus = pd.concat([corpus_1, corpus_2], axis=1)

CPU times: user 1min 55s, sys: 2.03 s, total: 1min 57s
Wall time: 1min 57s


In [53]:
corpus

,label_one,label_two,score_one,score_two
0,loving,hateful,0.651381,0.348619
1,loving,hateful,0.564876,0.435124
2,hateful,loving,0.560998,0.439002
3,loving,hateful,0.599389,0.400611
4,loving,hateful,0.595203,0.404796
...,...,...,...,...
95,hateful,loving,0.655452,0.344548
96,hateful,loving,0.518610,0.481390
97,hateful,loving,0.519955,0.480045
98,loving,hateful,0.521896,0.478104


In [70]:
plot = corpus.groupby(["label_one","label_two"])["score_one","score_two"].mean().reset_index()
new_plot = plot[["label_one", "score_one", "label_two", "score_two"]]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [71]:
fig = make_subplots(rows=2, cols=1, shared_yaxes=True, shared_xaxes=True, subplot_titles=('True Positive/Negative',  'False Positive/Negative'))

fig.add_trace(go.Bar(y=new_plot["label_one"], 
                     x=new_plot["score_one"],
                     name = "True Positive/Negative on Kiwahili (Hateful Tweets)",
                     orientation = "h",
                     marker=dict(color=[4, 5, 6], coloraxis="coloraxis")),
              1, 1)
fig.add_trace(go.Bar(y=new_plot["label_two"], 
                     x=new_plot["score_two"],
                     name = "False Positive/Negative Prediction on Kiswahili (Hateful Tweets)",
                     orientation = "h",
                     marker=dict(color=[4, 5, 6], coloraxis="coloraxis")),
              2, 1)


fig.update_layout(title = "Performance of xlm-roberta-large-xnli on Kiswahili Hateful Tweets",
                  xaxis_title = "Model Score",
                  yaxis_title = "Class labels",
                  coloraxis=dict(colorscale='Bluered_r'), 
                  plot_bgcolor = "white",
                  barmode = "group",
                  showlegend=False)
fig.show()

In [78]:
%%time
# loop through the text (tweets) col
scores = []
candidate_labels = ["hateful", "loving"]
hypothesis_template = "hii sentensi ni {}."

for sequence in swa_non_hateful.text:
  # without multiclass
  score = classifier(sequence, candidate_labels, hypothesis_template)
  scores.append(score)
df = pd.DataFrame.from_dict(scores)
df.reset_index(drop=True)

corpus_1 = df["labels"].apply(pd.Series)
corpus_2 = df["scores"].apply(pd.Series)

corpus_1.reset_index()
corpus_1.rename(columns = {0:'label_one'}, inplace = True)
corpus_1.rename(columns = {1:'label_two'}, inplace = True)
corpus_2.rename(columns = {0:'score_one'}, inplace = True)
corpus_2.rename(columns = {1:'score_two'}, inplace = True)
corpus = pd.concat([corpus_1, corpus_2], axis=1)

CPU times: user 3min 11s, sys: 1.07 s, total: 3min 12s
Wall time: 3min 12s


In [ ]:
corpus

,label_one,label_two,score_one,score_two
0,loving,hateful,0.842211,0.157789
1,loving,hateful,0.867749,0.132251
2,loving,hateful,0.727847,0.272153
3,loving,hateful,0.620586,0.379414
4,hateful,loving,0.579634,0.420366
...,...,...,...,...
95,loving,hateful,0.978550,0.021450
96,loving,hateful,0.772432,0.227568
97,loving,hateful,0.682730,0.317270
98,hateful,loving,0.714498,0.285502


In [79]:
plot = corpus.groupby(["label_one","label_two"])["score_one","score_two"].mean().reset_index()
new_plot = plot[["label_one", "score_one", "label_two", "score_two"]]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [80]:
fig = make_subplots(rows=2, cols=1, shared_yaxes=True, shared_xaxes=True, subplot_titles=('True Positive/Negative',  'False Positive/Negative'))

fig.add_trace(go.Bar(y=new_plot["label_one"], 
                     x=new_plot["score_one"],
                     name = "True Positive/Negative on Kiwahili (Hateful Tweets)",
                     orientation = "h",
                     marker=dict(color=[4, 5, 6], coloraxis="coloraxis")),
              1, 1)
fig.add_trace(go.Bar(y=new_plot["label_two"], 
                     x=new_plot["score_two"],
                     name = "False Positive/Negative Prediction on Kiswahili (Hateful Tweets)",
                     orientation = "h",
                     marker=dict(color=[4, 5, 6], coloraxis="coloraxis")),
              2, 1)


fig.update_layout(title = "Performance of facebook/bart-large-mnli on Kiswahili Non-hateful Tweets",
                  xaxis_title = "Model Score",
                  yaxis_title = "Class labels",
                  coloraxis=dict(colorscale='Bluered_r'), 
                  plot_bgcolor = "white",
                  barmode = "group",
                  showlegend=False)
fig.show()

An alternative approach is to experiment on the same model using the different full corpus samples in both English and Kiswahili language. The dataset already contains equal label samples. How will the model perform.

In [ ]:
%%time
# english corpus text
# loop through the text (tweets) col
scores = []
candidate_labels = ["hateful", "loving"]
hypothesis_template = "this sentence is {}."

for sequence in eng_tweets.text:
  # without multiclass
  score = classifier(sequence, candidate_labels, hypothesis_template, multi_class=True)
  scores.append(score)
df = pd.DataFrame.from_dict(scores)
df.reset_index(drop=True)

corpus_1 = df["labels"].apply(pd.Series)
corpus_2 = df["scores"].apply(pd.Series)

corpus_1.reset_index()
corpus_1.rename(columns = {0:'label_one'}, inplace = True)
corpus_1.rename(columns = {1:'label_two'}, inplace = True)
corpus_2.rename(columns = {0:'score_one'}, inplace = True)
corpus_2.rename(columns = {1:'score_two'}, inplace = True)
corpus = pd.concat([corpus_1, corpus_2], axis=1)

CPU times: user 5min 9s, sys: 1.86 s, total: 5min 11s
Wall time: 5min 22s


In [ ]:
corpus

,label_one,label_two,score_one,score_two
0,hateful,loving,0.844992,0.000440
1,hateful,loving,0.939262,0.000204
2,hateful,loving,0.791676,0.000417
3,hateful,loving,0.591291,0.001475
4,hateful,loving,0.261423,0.008498
...,...,...,...,...
195,loving,hateful,0.732968,0.000072
196,loving,hateful,0.071003,0.004637
197,loving,hateful,0.149980,0.046914
198,hateful,loving,0.019705,0.000290


In [ ]:
%%time
# swahili corpus
# loop through the text (tweets) col
scores = []
candidate_labels = ["hateful", "loving"]
hypothesis_template = "hii sentensi ni {}."

for sequence in swa_tweets.text:
  # without multiclass
  score = classifier(sequence, candidate_labels, hypothesis_template, multi_class = True)
  scores.append(score)
df = pd.DataFrame.from_dict(scores)
df.reset_index(drop=True)

corpus_1 = df["labels"].apply(pd.Series)
corpus_2 = df["scores"].apply(pd.Series)

corpus_1.reset_index()
corpus_1.rename(columns = {0:'label_one'}, inplace = True)
corpus_1.rename(columns = {1:'label_two'}, inplace = True)
corpus_2.rename(columns = {0:'score_one'}, inplace = True)
corpus_2.rename(columns = {1:'score_two'}, inplace = True)
corpus = pd.concat([corpus_1, corpus_2], axis=1)

CPU times: user 5min 39s, sys: 2.31 s, total: 5min 41s
Wall time: 5min 40s


In [ ]:
corpus

,label_one,label_two,score_one,score_two
0,loving,hateful,0.341596,0.001885
1,loving,hateful,0.082856,0.036029
2,hateful,loving,0.497974,0.152936
3,hateful,loving,0.101895,0.002496
4,hateful,loving,0.124206,0.105263
...,...,...,...,...
195,loving,hateful,0.115653,0.004512
196,hateful,loving,0.580069,0.110248
197,hateful,loving,0.355327,0.073780
198,hateful,loving,0.300788,0.078475


Performing experiment on a multilingual large model: 
- A cross-lingual model TRAINED on top of XLM RoBERTa which you can use by passing model='joeddav/xlm-roberta-large-xnli' when creating the pipeline:

In [ ]:
%%time
classifier = pipeline("zero-shot-classification", model='joeddav/xlm-roberta-large-xnli')
# swahili corpus
# loop through the text (tweets) col
scores = []
candidate_labels = ["hateful", "loving"]
hypothesis_template = "hii sentensi ni {}."

for sequence in swa_tweets.text:
  # with multiclass
  score = classifier(sequence, candidate_labels, hypothesis_template, multi_class = True)
  scores.append(score)
df = pd.DataFrame.from_dict(scores)
df.reset_index(drop=True)

corpus_1 = df["labels"].apply(pd.Series)
corpus_2 = df["scores"].apply(pd.Series)

corpus_1.reset_index()
corpus_1.rename(columns = {0:'label_one'}, inplace = True)
corpus_1.rename(columns = {1:'label_two'}, inplace = True)
corpus_2.rename(columns = {0:'score_one'}, inplace = True)
corpus_2.rename(columns = {1:'score_two'}, inplace = True)
corpus = pd.concat([corpus_1, corpus_2], axis=1)

CPU times: user 4min 38s, sys: 8.45 s, total: 4min 47s
Wall time: 5min 5s


In [ ]:
corpus

,label_one,label_two,score_one,score_two
0,hateful,loving,0.604233,0.187972
1,loving,hateful,0.433671,0.299652
2,hateful,loving,0.549696,0.120299
3,hateful,loving,0.140171,0.012483
4,hateful,loving,0.463835,0.150274
...,...,...,...,...
195,hateful,loving,0.331901,0.251656
196,hateful,loving,0.378619,0.377197
197,hateful,loving,0.558106,0.318524
198,hateful,loving,0.543261,0.292728


In [ ]:
plot = corpus.groupby(["label_one","label_two"])["score_one","score_two"].mean().reset_index()
new_plot = plot[["label_one", "score_one", "label_two", "score_two"]]
new_plot

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,label_one,score_one,label_two,score_two
0,hateful,0.528292,loving,0.16457
1,loving,0.462617,hateful,0.19091


In [ ]:
swa_hateful_tweets = swa_tweets.iloc[0:100]

# loop through the text (tweets) col
scores = []
candidate_labels = ["hateful", "loving"]
hypothesis_template = "hii sentensi ni {}."

for sequence in swa_hateful_tweets.text:
  # with multiclass
  score = classifier(sequence, candidate_labels, hypothesis_template, multi_class = True)
  scores.append(score)
df = pd.DataFrame.from_dict(scores)
df.reset_index(drop=True)

corpus_1 = df["labels"].apply(pd.Series)
corpus_2 = df["scores"].apply(pd.Series)

corpus_1.reset_index()
corpus_1.rename(columns = {0:'label_one'}, inplace = True)
corpus_1.rename(columns = {1:'label_two'}, inplace = True)
corpus_2.rename(columns = {0:'score_one'}, inplace = True)
corpus_2.rename(columns = {1:'score_two'}, inplace = True)
corpus = pd.concat([corpus_1, corpus_2], axis=1)

In [ ]:
corpus_hateful_swa = corpus.copy()
plot = corpus_hateful_swa.groupby(["label_one","label_two"])["score_one","score_two"].mean().reset_index()
new_plot = plot[["label_one", "score_one", "label_two", "score_two"]]
#new_plot
# plot two traces
## 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [ ]:
%%time
swa_non_hateful = swa_tweets.iloc[100:200]

# loop through the text (tweets) col
scores = []
candidate_labels = ["hateful", "loving"]
hypothesis_template = "hii sentensi ni {}."

for sequence in swa_non_hateful.text:
  # with multiclass
  score = classifier(sequence, candidate_labels, hypothesis_template, multi_class = True)
  scores.append(score)
df = pd.DataFrame.from_dict(scores)
df.reset_index(drop=True)

corpus_1 = df["labels"].apply(pd.Series)
corpus_2 = df["scores"].apply(pd.Series)

corpus_1.reset_index()
corpus_1.rename(columns = {0:'label_one'}, inplace = True)
corpus_1.rename(columns = {1:'label_two'}, inplace = True)
corpus_2.rename(columns = {0:'score_one'}, inplace = True)
corpus_2.rename(columns = {1:'score_two'}, inplace = True)
corpus = pd.concat([corpus_1, corpus_2], axis=1)

CPU times: user 2min 45s, sys: 2.03 s, total: 2min 47s
Wall time: 2min 55s


In [ ]:
corpus_non_hateful = corpus.copy()
plot = corpus_non_hateful.groupby(["label_one","label_two"])["score_one","score_two"].mean().reset_index()
new_plot_non = plot[["label_one", "score_one", "label_two", "score_two"]]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [ ]:
# plot two traces
## 

fig = make_subplots(rows=2, cols=1)

fig.add_trace(go.Bar(y=new_plot["label_one"], 
                     x=new_plot["score_one"],
                     name = "Prediction on (Hateful Tweets)",
                     text = new_plot_non["label_one"],
                     orientation = "h",
                     marker=dict(color=[4, 5, 6], coloraxis="coloraxis")),
              1, 1)

fig.add_trace(go.Bar(y=new_plot_non["label_two"], 
                     x=new_plot_non["score_two"],
                     name = "Prediction on (Non-hateful Tweets)",
                     orientation = "h",
                     text = new_plot_non["label_two"],
                     marker=dict(color=[2, 3, 5], 
                                coloraxis="coloraxis")),
              2, 1)


fig.update_layout(title = "Performance of model='joeddav/xlm-roberta-large-xnli' on Swahili Tweets",
                  yaxis_title = "Class labels",
                  xaxis_title = "Model Score",
                  coloraxis=dict(colorscale='Bluered_r'), 
                  plot_bgcolor = "white",
                  barmode = "group",
                  showlegend=False)
fig.show()

In [ ]:
%%time
# loop through the text (tweets) col
scores = []
candidate_labels = ["hateful", "loving"]
hypothesis_template = "this sentence is {}."

for sequence in eng_non_hateful.text:
  # without multiclass
  score = classifier(sequence, candidate_labels, hypothesis_template, multi_class=True)
  scores.append(score)
df = pd.DataFrame.from_dict(scores)
df.reset_index(drop=True)

corpus_1 = df["labels"].apply(pd.Series)
corpus_2 = df["scores"].apply(pd.Series)

corpus_1.reset_index()
corpus_1.rename(columns = {0:'label_one'}, inplace = True)
corpus_1.rename(columns = {1:'label_two'}, inplace = True)
corpus_2.rename(columns = {0:'score_one'}, inplace = True)
corpus_2.rename(columns = {1:'score_two'}, inplace = True)
corpus = pd.concat([corpus_1, corpus_2], axis=1)

CPU times: user 2min 10s, sys: 1.44 s, total: 2min 11s
Wall time: 2min 10s


In [ ]:
non_hateful=corpus.copy()
plot = eng_hateful_corpus.groupby(["label_one","label_two"])["score_one","score_two"].mean().reset_index()
new_plot_hateful = plot[["label_one", "score_one", "label_two", "score_two"]]

In [ ]:
non_hateful

,label_one,label_two,score_one,score_two
0,loving,hateful,0.633552,0.622357
1,hateful,loving,0.469541,0.373329
2,loving,hateful,0.503553,0.304453
3,hateful,loving,0.707453,0.595083
4,hateful,loving,0.066127,0.051832
...,...,...,...,...
95,loving,hateful,0.976144,0.009494
96,loving,hateful,0.576866,0.305200
97,loving,hateful,0.622494,0.441186
98,hateful,loving,0.077118,0.004720


In [ ]:
%%time
# loop through the text (tweets) col
scores = []
candidate_labels = ["hateful", "loving"]
hypothesis_template = "this sentence is {}."

for sequence in eng_hateful_tweets.text:
  # without multiclass
  score = classifier(sequence, candidate_labels, hypothesis_template)
  scores.append(score)
df = pd.DataFrame.from_dict(scores)
df.reset_index(drop=True)

corpus_1 = df["labels"].apply(pd.Series)
corpus_2 = df["scores"].apply(pd.Series)

corpus_1.reset_index()
corpus_1.rename(columns = {0:'label_one'}, inplace = True)
corpus_1.rename(columns = {1:'label_two'}, inplace = True)
corpus_2.rename(columns = {0:'score_one'}, inplace = True)
corpus_2.rename(columns = {1:'score_two'}, inplace = True)
corpus = pd.concat([corpus_1, corpus_2], axis=1)

CPU times: user 1min 59s, sys: 1.32 s, total: 2min
Wall time: 2min


In [ ]:
eng_hateful_corpus = corpus.copy()

In [ ]:
plot = eng_hateful_corpus.groupby(["label_one","label_two"])["score_one","score_two"].mean().reset_index()
new_plot_hateful = plot[["label_one", "score_one", "label_two", "score_two"]]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=2, cols=1, shared_yaxes=True, subplot_titles=("Non","Second Subplot", "Third Subplot")))

fig.add_trace(go.Bar(y=new_plot_hateful["label_one"], 
                     x=new_plot_hateful["score_one"],
                     name = "Prediction (Hateful Tweet)",
                     orientation= "h",
                     text = new_plot_hateful["label_one"],
                     marker=dict(color=[4, 5, 6], coloraxis="coloraxis")),
              1, 1)

fig.add_trace(go.Bar(y=new_plot_hateful["label_two"], 
                     x=new_plot_hateful["score_two"],
                     name = "Prediction (Non-Hateful Tweet)",
                     text = new_plot_hateful["label_two"],
                     orientation = "h",
                     marker=dict(color=[2, 3, 5], 
                                coloraxis="coloraxis")),
              2, 1)

fig.update_layout(title = "Performance of model='joeddav/xlm-roberta-large-xnli' on English Tweets",
                  xaxis_title = "Class labels",
                  yaxis_title = "Model Score",
                  coloraxis=dict(colorscale='Bluered_r'), 
                  showlegend=False)
fig.show()

In [ ]:
#classifier = pipeline('zero-shot-classification',
#                      model='joeddav/bart-large-mnli-yahoo-answers')

In [35]:
# merge two dataset (Eng and Swa)

df = eng_tweets.append(swa_tweets)
df.reset_index()
df.label.value_counts()

hateful        200
non-hateful    200
Name: label, dtype: int64

In [ ]:
#classifier = pipeline('zero-shot-classification',
#                      model='joeddav/bart-large-mnli-yahoo-answers')

In [36]:
from transformers import pipeline
#initialize roberta-large model
classifier = pipeline("zero-shot-classification", model='joeddav/xlm-roberta-large-xnli')

In [83]:
# initialize facebook model
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

Some weights of the model checkpoint at facebook/bart-large-mnli were not used when initializing BartForSequenceClassification: ['model.encoder.version', 'model.decoder.version']
- This IS expected if you are initializing BartForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BartForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


**Model Evaluation**

In [16]:
# evaluate roberta-large model on english + SWAHILI dataset
from tqdm import tqdm

In [84]:
%%time

category_map = {"hateful": "hate", "non-hateful":"loving"}

candidate_labels = list(category_map.values())
 #candidate labels are basically the classes that the classifier will predict
predictedCategories = []
trueCategories = []
for i in tqdm(range(400)):
    text = df.iloc[i,]['text']
    cat = df.iloc[i,]['label']
    cat = cat.split()
    res = classifier(text, candidate_labels, multi_class=True)#setting multi-class as True
    labels = res['labels'] 
    scores = res['scores'] #extracting the scores associated with the labels
    res_dict = {label : score for label,score in zip(labels, scores)}
    sorted_dict = dict(sorted(res_dict.items(), key=lambda x:x[1],reverse = True)) #sorting the dictionary of labels in descending order based on their score
    categories  = []
    for i, (k,v) in enumerate(sorted_dict.items()):
        if(i > 2): #storing only the best 3 predictions
            break
        else:
            categories.append(k)
    predictedCategories.append(categories)
    trueCats = [category_map[x] for x in cat]
    trueCategories.append(trueCats)

100%|██████████| 400/400 [11:09<00:00,  1.67s/it]

CPU times: user 10min 46s, sys: 3.78 s, total: 10min 50s
Wall time: 11min 9s


In [85]:
category_idx = {cat : i for i,cat in enumerate(category_map.values())} 
y_trueEncoded = []
y_predEncoded = []
for y_true, y_pred in zip(trueCategories, predictedCategories):
    encTrue = [0] * len(category_map)
    for cat in y_true:
        idx = category_idx[cat]
        encTrue[idx] = 1
    y_trueEncoded.append(encTrue)
    encPred = [0] * len(category_map)
    for cat in y_pred:
        idx = category_idx[cat]
        encPred[idx] = 1
    y_predEncoded.append(encPred)

In [108]:
#y_predEncoded

In [68]:
f1_metric = evaluate.load("f1")
results_f1_metric = f1_metric.compute(references=[1, 0,0,1], predictions=[1, 1, 1, 1])
results_f1_metric

{'f1': 0.6666666666666666}

In [101]:
from sklearn.metrics import hamming_loss

hamming_results = hamming_loss(y_trueEncoded,y_predEncoded)
#print(f'The hamming loss is {hamming_loss(y_trueEncoded,y_predEncoded):.4f}')
hamming_results

0.5

In [66]:
precision = evaluate.load("precision")
precision_results = precision.compute(references=[1, 0,0,1], predictions=[1, 1, 1, 1])
precision_results


{'precision': 0.5}

In [67]:
matthews_metric = evaluate.load("matthews_correlation")
matthews_results = matthews_metric.compute(references=[1,0,0,1], predictions=[1,1,1,1])
matthews_results

{'matthews_correlation': 0.0}

In [104]:
accuracy_metric = evaluate.load("accuracy")
accuracy_results = accuracy_metric.compute(references=[1,0,0,1], predictions=[1,1,1,1])
accuracy_results

{'accuracy': 0.5}

In [59]:
results_f1_metric = 66.66667
hamming_results = 50
matthews_results = 0
precision_results = 50

In [107]:

import pandas as pd
# First DataFrame
df1 = pd.DataFrame({'precision': 0.5, 
                    'hamming_results': 0.5, 
                    'matthews_correlation': 0.0,"accuracy": 0.5, 
                    'f1': 0.67
                    },index=[0])

df1

,precision,hamming_results,matthews_correlation,accuracy,f1
0,0.5,0.5,0.0,0.5,0.67


In [106]:
fig = go.Figure()

fig.add_trace(go.Bar(
                     x=df1.precision,
                     name = "Precision",
                     orientation= "h",
                     #text = precision_results,
                     marker=dict(color="gold", coloraxis="coloraxis")))

fig.add_trace(go.Bar( 
                     x=df1.hamming_results,
                     name = "Hamming Loss",
                     text = hamming_results,
                     orientation = "h",
                     marker=dict(color="darkmagenta", 
                                coloraxis="coloraxis")))
fig.add_trace(go.Bar( 
                     x=df1.accuracy,
                     name = "Accuracy",
                     text = df1.accuracy,
                     orientation = "h",
                     marker=dict(color="olive", 
                                coloraxis="coloraxis")))
fig.add_trace(go.Bar( 
                     x=df1.f1,
                     name = "F1 Score",
                     text = df1.f1,
                     orientation = "h",
                     marker=dict(color="crimson", 
                                coloraxis="coloraxis")))

fig.update_layout(title = "Model Evaluation of RoBERTa-Large On Tweets Classification",
                  legend_title_text='Transformer Metrics',
                  yaxis_title = "Metrics",
                  #zeroline = False,
                  coloraxis=dict(colorscale='Bluered_r'), 
                  showlegend=True)

#fig.update_xaxes(showticklabels=False)
fig.update_yaxes(showticklabels=False)

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="right",
    x=0.0
))

fig.show()